In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
import fiona 
import geopandas as gpd 

In [ ]:
path = os.path.dirname(os.path.abspath(os.path.dirname(__name__)))
frequencia_atendimento = path+"/arquivos/frequencia_atendimento.xlsx"
quantidade_onibus = path+"/arquivos/viagens.xlsx"

df_frequencia = pd.read_excel(frequencia_atendimento)
df_quantidade = pd.read_excel(quantidade_onibus)

df_frequencia['Linha'] = df_frequencia['Linha'].astype(str)

# print(len(df_frequencia['Linha']))
df_frequencia.sort_values('Frequência')

freq_otima, freq_boa, freq_suficiente, freq_insuficiente = 10, 15, 30, 40

In [ ]:
df_quantidade['linha'] = df_quantidade['linha'].astype(str)
df_quantidade['quantidade_veiculos'] = df_quantidade['quantidade_veiculos'].astype(int)
df_quantidade = df_quantidade[['linha','quantidade_veiculos']]
df_quantidade = df_quantidade.drop(25)
df_quantidade.sort_values('linha', ascending=True)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 6))

ax.bar(df_quantidade['linha'], df_quantidade['quantidade_veiculos'])

ax.set_title('Quantidade de veículos x linha de ônibus')
ax.set_xlabel('Linha de Ônibus')
ax.set_ylabel('Quantidade de veículos')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(18, 6))

ax.plot(df_frequencia['Linha'], df_frequencia['Frequência'], 'bo')

ax.axhline(freq_otima, color='orange', linestyle='--', label=f'Frequência Ótima: {freq_otima}')
ax.axhline(freq_boa, color='r', linestyle='--', label=f'Frequência Boa: {freq_boa}')
ax.axhline(freq_suficiente, color='green', linestyle='--', label=f'Frequência Suficiente: {freq_suficiente}')
ax.axhline(freq_insuficiente, color='purple', linestyle='--', label=f'Frequência Insuficiente: {freq_insuficiente}')

ax.set_title('Frequência de atendimento das Linhas de Ônibus', fontdict = {'fontsize': 24})
ax.set_xlabel('Linha de Ônibus', fontdict = {'fontsize': 12})
ax.set_ylabel('Frequência atendimento (min)', fontdict = {'fontsize': 18})
ax.legend(bbox_to_anchor=(0.5, -0.1), loc='upper center', borderaxespad=0., ncol=4, fontsize = 14)
plt.show()

In [ ]:
ciclo_viagem = path+"/arquivos/planilha_gtfs.xlsx"
df_ciclo = pd.read_excel(ciclo_viagem, sheet_name='operacao')
df_ciclo_correto = df_ciclo[df_ciclo['sentido'] == 'ida']
df_ciclo = df_ciclo_correto[['linha', 'intervalo_minutos', 'ciclo_viagem']]

df_ciclo = df_ciclo.dropna()
df_ciclo['linha'] = df_ciclo['linha'].astype(str)

df_ciclo

In [ ]:
media_ciclo = sum(df_ciclo['ciclo_viagem'])/len(df_ciclo['ciclo_viagem'])

In [ ]:

fig, ax = plt.subplots(figsize=(20, 6))

ax.bar(df_ciclo['linha'], df_ciclo['ciclo_viagem'])

ax.axhline(media_ciclo, color='red', linestyle='--', label=f'Média ciclo de viagem: {media_ciclo:.2f}')

ax.set_title('Ciclo de viagem das Linhas de Ônibus')
ax.set_xlabel('Linha de Ônibus')
ax.set_ylabel('Ciclo de viagem (minutos)')
ax.legend(loc='upper left')
plt.show()

In [ ]:
indicadores_prioridades = {
    'nomeclatura': ['I4', 'I1', 'I2', 'I3', 'I5', 'I7', 'I6', 'I8', 'I9', 'I10'],
    'prioridade': [0.2269, 0.1526, 0.1121, 0.0997, 0.0992, 0.0954, 0.0831, 0.0756, 0.0277, 0.0277],
    'indicador': ['Pontualidade – cumprir horários', 'Porcentagem das vias pavimentadas', 'Distância entre pontos', 'Integração municipal do sistema de transporte', 'Frequência de atendimento', 'Abrangência da rede – atender a cidade', 'Cumprimento dos itinerários', 'Treinamento e capacitação dos motoristas', 'Existência Sistema de informação pela internet', 'Valor da Tarifa '],
}
df_indicadores_prioridades = pd.DataFrame(indicadores_prioridades)

indicadores_para_usar = ['I4', 'I5', 'I6']
df_indicadores_prioridades

In [ ]:
df_indicadores_para_usar = df_indicadores_prioridades[df_indicadores_prioridades['nomeclatura'].isin(indicadores_para_usar)]
df_indicadores_para_usar

In [ ]:
pontuacao_indicadores = {
    'linha' : [],
    'I4' : [],
    'I5': [],
    'I6': [],
}

print(len(df_ciclo))

for row in df_ciclo.itertuples():
    pontuacao_indicadores['linha'].append(row.linha)
    if freq_otima <= row.intervalo_minutos < freq_boa:
        pontuacao = 3
    elif freq_boa <= row.intervalo_minutos < freq_suficiente:
        pontuacao = 2
    elif freq_suficiente <= row.intervalo_minutos < freq_insuficiente:
        pontuacao = 1
    else:
        pontuacao = 0
    pontuacao_indicadores['I5'].append(pontuacao)
    # print(row)
pontuacao_indicadores['I4'] = [0] * len(df_ciclo)
pontuacao_indicadores['I6'] = [0] * len(df_ciclo)
df_pontuacao_indicadores = pd.DataFrame(pontuacao_indicadores)

df_pontuacao_indicadores['linha']

In [ ]:

path = os.path.dirname(os.path.abspath(os.path.dirname(__name__)))

fiona.drvsupport.supported_drivers['libkml'] = 'rw' 
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'

path = path + "/arquivos/"
kmz_file_path = path + "linhas_observatório.kmz"
kml_file_path = path + "aaaaaaa.kml"

layers = gpd.list_layers(kml_file_path)

gdf_list = []
for layer in fiona.listlayers(kml_file_path):
    if layer == 'Linhas prontas':
        continue
    gdf = gpd.read_file(kml_file_path, driver='LIBKML', layer=layer)
    gdf_list.append(gdf)

gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

gdf_filtered = gdf.query("Name != ''")
gdf_filtered = gdf_filtered.query("Name != '1'")
gdf = gdf_filtered.query("Name != 'inativo'")
gdf['Description'] = gdf['Description'].str.replace(',', '.')

gdf_ida = gdf[gdf['Name'].str.contains('IDA', case=False, na=False)]
gdf_ida['Name'] = gdf['Name'].str.replace(' - IDA', "")

gdf_ida['Description'] = gdf_ida['Description'].astype(float)

In [ ]:
gdf_ida['Name'] = gdf_ida['Name'].str.replace('- IDA', '')
gdf_ida = gdf_ida[gdf_ida['Name'] != '6604']

In [ ]:
gdf_ida = gdf_ida.reset_index()

gdf_ida


In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))

ax.bar(df_quantidade['linha'], df_quantidade['quantidade_veiculos']/gdf_ida['Description'])

ax.set_title('Quantidade de veículos por KM x Linha de Ônibus')
ax.set_xlabel('Linha de Ônibus')
ax.set_ylabel('Quantidade de veículos por KM')
ax.legend(loc='upper left')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 6))
media_km = sum(gdf_ida['Description'])/len(gdf_ida['Description'])
ax.bar(gdf_ida['Name'], gdf_ida['Description'])

ax.axhline(media_km, color='red', linestyle='--', label=f'Média KM percorrido: {media_km:.2f}')

ax.set_title('Quantidade de KM percorridos x Linha de Ônibus')
ax.set_xlabel('Linha de Ônibus')
ax.set_ylabel('Quantidade de KM percorridos')
ax.legend(loc='upper left')
plt.show()